# Federateed Learning

### Prerequisites
- terraform
- python3
- aws cli (configured)
- bacalhau cli

In [ ]:
%cd multi-region

## Add your regions to regions.md
### Note: The Bootstrap region should be first

In [2]:
%cat regions.md

# aws ec2 describe-regions | jq -r '.Regions[].RegionName'
# Bootstrap region must be first
ca-central-1
eu-north-1
us-east-1
us-west-1


### To Get AMI IDs for each region

In [ ]:
#Replace the value of the name parameter with the name of your AMI
%python3 get_instance_amis.py --name "Deep Learning AMI GPU PyTorch 2.0.1 (Ubuntu 20.04) 20230620"

{
    "ca-central-1": {
        "region": "ca-central-1",
        "zone": "ca-central-1a",
        "instance_ami": "ami-0c483e1cc1d6e27f8"
    },
    "eu-north-1": {
        "region": "eu-north-1",
        "zone": "eu-north-1a",
        "instance_ami": "ami-033855fe5b2db3d01"
    },
    "us-east-1": {
        "region": "us-east-1",
        "zone": "us-east-1a",
        "instance_ami": "ami-0240b7264c1c9e6a9"
    },
    "us-west-1": {
        "region": "us-west-1",
        "zone": "us-west-1a",
        "instance_ami": "ami-0eaef2301ab069ace"
    }
}


## Replace the Values of "locations" with the values outputted from the above command

In [10]:
%cat ./tf/.env.example

{
  "access_key": "AKIAUEMPQ7JFEXAMPLE",
  "secret_key": "a3YhVb06muc4+YSnlzqLzJJ2sOrxl/EXAMPLE",
  "bootstrap_region": "ca-central-1", # Bootstrap region must be firts (also, delete this line in the final, json doesn't support comments)
  "locations": {
    "ca-central-1": {
      "region": "ca-central-1",
      "availability_zone": "ca-central-1a",
      "instance_ami": "ami-0ea18256de20ecdfc"
    },
    "us-west-2": {
      "region": "us-west-2",
      "availability_zone": "us-west-2a",
      "instance_ami": "ami-0fcf52bcf5db7b003"
    },
    "eu-west-1": {
      "region": "eu-west-1",
      "availability_zone": "eu-west-1a",
      "instance_ami": "ami-00aa9d3df94c6c354"
    },
    "eu-north-1": {
      "region": "eu-north-1",
      "availability_zone": "eu-north-1a",
      "instance_ami": "ami-0a79730daaf45078a"
    },
    "ap-southeast-1": {
      "region": "ap-southeast-1",
      "availability_zone": "ap-southeast-1a",
      "instance_ami": "ami-0a72af05d27b49ccb"
    }
  },
  "a

## Replace The values of the keys with your own key values and also the value of instance_type if you want

In [ ]:
%%bash
cp -r .env.example .env.json

### Deploying the infra

In [ ]:
%%bash
./bulk-deploy.sh create

#### Building the container (Optional)

##### Install Git-LFS
```
sudo apt-get install git-lfs
git lfs install
```

##### Clone the repo and follow the instructions in the readme to build your container
```
git clone https://huggingface.co/VedantPadwal/federated/
```

### Running the jobs

### 

There are two jobs we will run
- Job To generate the gradients and save them to S3
- Job To Update the Model with the gradients and save it to S3

In [ ]:
!cat tf/bacalhau.run

export BACALHAU_IPFS_SWARM_ADDRESSES=/ip4/100.109.253.101/tcp/45785/p2p/Qmaf97J99ES8qSTJ8t8NqFmMhm1fwVUQU3GhNNNBvsFPAp
export BACALHAU_API_HOST=0.0.0.0
export BACALHAU_API_PORT=1234
export BACALHAU_PEER_CONNECT=/ip4/100.109.253.101/tcp/40259/p2p/QmS88zXzGW2TFXsCTnt2yW8LvLxjvRSsnGaH61efBYkCAT


##### Update BACALHAU_API_HOST with Public IP of the Bootstrap Node

Paste the values and execute the cell

In [ ]:
%%bash
export BACALHAU_IPFS_SWARM_ADDRESSES=/ip4/100.109.253.101/tcp/45785/p2p/Qmaf97J99ES8qSTJ8t8NqFmMhm1fwVUQU3GhNNNBvsFPAp
export BACALHAU_API_HOST=238.69.113.41
export BACALHAU_API_PORT=1234
export BACALHAU_PEER_CONNECT=/ip4/100.109.253.101/tcp/40259/p2p/QmS88zXzGW2TFXsCTnt2yW8LvLxjvRSsnGaH61efBYkCAT

## Generating the Gradients

In [ ]:
%%bash
python3 generate_gradients.py

## Update Model with the Gradients

In [ ]:
%%bash
python3 update_model.py

S3 URL to the model: https://s3.console.aws.amazon.com/s3/object/multiregion-bacalhau-s3-inference-us-east-1-o-images-bucket?region=us-east-1&prefix=*/outputs/brain_tumor_classifier_updated.h5
